In [1]:
from __future__ import print_function

import numpy as np
from torch.utils import data
import argparse
import torch
import torch.utils.data as data_utils
import torch.optim as optim
from torch.autograd import Variable
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
import math
seed = 11
torch.manual_seed(seed)
torch.cuda.manual_seed(0)
np.random.seed(seed)

# torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    numpy.random.seed(worker_seed)
    random.seed(worker_seed)
    
    
class Dataset(data.Dataset):
    def __init__(self, X1,Y):
        self.X1 = X1
        self.Y = Y
    def __len__(self):        
        return len(self.X1)
    
    def __getitem__(self, index):
        x = self.X1[index]
        y = self.Y[index]
        return x,y

#from model import Attention, GatedAttention

## Negetive pictures

In [2]:
xTrain=np.load("../extracted_features/FS/input_features_free_train.npy")

yTrain=np.load("../extracted_features/FS/label_free_train.npy")

xTest=np.load("../extracted_features/FS/input_features_free_test.npy")

yTest=np.load("../extracted_features/FS/label_free_test.npy")

xVal=np.load("../extracted_features/FS/input_features_free_val.npy")

yVal=np.load("../extracted_features/FS/label_free_val.npy")

xTrain = xTrain.reshape(len(xTrain),1,19,24)
xTest = xTest.reshape(len(xTest),1,19,24)
xVal = xVal.reshape(len(xVal),1,19,24)

print(xTrain.shape,yTrain.shape)
print(xTest.shape,yTest.shape)
print(xVal.shape,yVal.shape)


traindataset = Dataset(xTrain,yTrain)
testdataset = Dataset(xTest,yTest)
valdataset = Dataset(xVal,yVal)


(1121, 1, 19, 24) (1121,)
(190, 1, 19, 24) (190,)
(199, 1, 19, 24) (199,)


In [3]:
class Stammering(data_utils.Dataset):
    def __init__(self, target_number=1, mean_bag_length=5, var_bag_length=2, num_bag=150, seed=2021, train="train"):
        self.target_number = target_number
        self.mean_bag_length = mean_bag_length
        self.var_bag_length = var_bag_length
        self.num_bag = num_bag
        self.train = train
        self.r = np.random.RandomState(seed)

        if self.train=="train":
            self.train_bags_list, self.train_labels_list = self._create_bags()
        elif self.train=="val":
            self.val_bags_list, self.val_labels_list = self._create_bags()
        else:
            self.test_bags_list, self.test_labels_list = self._create_bags()

    def _create_bags(self):
        if self.train=="train":
            print("train")
            loader = data_utils.DataLoader(traindataset,
                                           batch_size=1,
                                           shuffle=True)
        elif self.train=="val":
            print("val")
            loader = data_utils.DataLoader(valdataset,
                                           batch_size=1,
                                           shuffle=True)
        else:
            loader = data_utils.DataLoader(testdataset,
                                           batch_size=1,
                                           shuffle=True)
            
        bags_list = []
        labels_list = []
        for (batch_data, batch_labels) in loader:
            #print(batch_data.shape)
            bags_list.append(batch_data.reshape(19,1,24))
            temp = torch.as_tensor(np.array([batch_labels for x in range(19)]))
            labels_list.append(temp)
            
               
        #print(bags_list)
        #print(labels_list)
        return bags_list, labels_list

    def __len__(self):
        if self.train=="train":
            return len(self.train_labels_list)
        elif self.train=="val":
            return len(self.val_labels_list)
        else:
            return len(self.test_labels_list)

    def __getitem__(self, index):
        if self.train=="train":
            bag = self.train_bags_list[index]
            label = [max(self.train_labels_list[index]), self.train_labels_list[index]]
        elif self.train=="val":
            bag = self.val_bags_list[index]
            label = [max(self.val_labels_list[index]), self.val_labels_list[index]]
        else:
            bag = self.test_bags_list[index]
            label = [max(self.test_labels_list[index]), self.test_labels_list[index]]

        return bag, label

In [4]:
train_loader = data_utils.DataLoader(Stammering(train="train"),batch_size=1,worker_init_fn=seed_worker,shuffle=True)
test_loader = data_utils.DataLoader(Stammering(train="val"),batch_size=1,worker_init_fn=seed_worker,shuffle=True)
val_loader = data_utils.DataLoader(Stammering(train="test"),batch_size=1,worker_init_fn=seed_worker,shuffle=True)

train
val


In [5]:
class _NonLocalBlockND(nn.Module):
    def __init__(self, in_channels, inter_channels=None, dimension=3, sub_sample=True, bn_layer=True):
        super(_NonLocalBlockND, self).__init__()

        assert dimension in [1, 2, 3]

        self.dimension = dimension
        self.sub_sample = sub_sample

        self.in_channels = in_channels
        self.inter_channels = inter_channels

        if self.inter_channels is None:
            self.inter_channels = in_channels // 2
            if self.inter_channels == 0:
                self.inter_channels = 1

        if dimension == 3:
            conv_nd = nn.Conv3d
            max_pool_layer = nn.MaxPool3d(kernel_size=(1, 2, 2))
            bn = nn.BatchNorm3d
        elif dimension == 2:
            conv_nd = nn.Conv2d
            max_pool_layer = nn.MaxPool2d(kernel_size=(2, 2))
            bn = nn.BatchNorm2d
        else:
            conv_nd = nn.Conv1d
            max_pool_layer = nn.MaxPool1d(kernel_size=(2))
            bn = nn.BatchNorm1d

        self.g = conv_nd(in_channels=self.in_channels, out_channels=self.inter_channels,
                         kernel_size=1, stride=1, padding=0)

        if bn_layer:
            self.W = nn.Sequential(
                conv_nd(in_channels=self.inter_channels, out_channels=self.in_channels,
                        kernel_size=1, stride=1, padding=0),
                bn(self.in_channels)
            )
            nn.init.constant(self.W[1].weight, 0)
            nn.init.constant(self.W[1].bias, 0)
        else:
            self.W = conv_nd(in_channels=self.inter_channels, out_channels=self.in_channels,
                             kernel_size=1, stride=1, padding=0)
            nn.init.constant(self.W.weight, 0)
            nn.init.constant(self.W.bias, 0)

        self.theta = conv_nd(in_channels=self.in_channels, out_channels=self.inter_channels,
                             kernel_size=1, stride=1, padding=0)
        self.phi = conv_nd(in_channels=self.in_channels, out_channels=self.inter_channels,
                           kernel_size=1, stride=1, padding=0)

        if sub_sample:
            self.g = nn.Sequential(self.g, max_pool_layer)
            self.phi = nn.Sequential(self.phi, max_pool_layer)

    def forward(self, x):
        '''
        :param x: (b, c, t, h, w)
        :return:
        '''

        batch_size = x.size(0)

        g_x = self.g(x).view(batch_size, self.inter_channels, -1)
        g_x = g_x.permute(0, 2, 1)

        theta_x = self.theta(x).view(batch_size, self.inter_channels, -1)
        theta_x = theta_x.permute(0, 2, 1)
        phi_x = self.phi(x).view(batch_size, self.inter_channels, -1)
        f = torch.matmul(theta_x, phi_x)
        f_div_C = F.softmax(f, dim=-1)

        y = torch.matmul(f_div_C, g_x)
        y = y.permute(0, 2, 1).contiguous()
        y = y.view(batch_size, self.inter_channels, *x.size()[2:])
        W_y = self.W(y)
        z = W_y + x

        return z


class NONLocalBlock1D(_NonLocalBlockND):
    def __init__(self, in_channels, inter_channels=None, sub_sample=True, bn_layer=True):
        super(NONLocalBlock1D, self).__init__(in_channels,
                                              inter_channels=inter_channels,
                                              dimension=1, sub_sample=sub_sample,
                                              bn_layer=bn_layer)
class NONLocalBlock2D(_NonLocalBlockND):
    def __init__(self, in_channels, inter_channels=None, sub_sample=True, bn_layer=True):
        super(NONLocalBlock2D, self).__init__(in_channels,
                                              inter_channels=inter_channels,
                                              dimension=2, sub_sample=sub_sample,
                                              bn_layer=bn_layer)
# class NonLocalBlock(nn.Module):
#     def __init__(self, channel):
#         super(NonLocalBlock, self).__init__()
#         self.inter_channel = channel
#         self.conv_phi = nn.Conv1d(in_channels=channel, out_channels=self.inter_channel, kernel_size=1, stride=1,padding=0, bias=False)
#         self.conv_theta = nn.Conv1d(in_channels=channel, out_channels=self.inter_channel, kernel_size=1, stride=1, padding=0, bias=False)
#         self.conv_g = nn.Conv1d(in_channels=channel, out_channels=self.inter_channel, kernel_size=1, stride=1, padding=0, bias=False)
#         self.softmax = nn.Softmax(dim=1)
#         self.conv_mask = nn.Conv1d(in_channels=self.inter_channel, out_channels=channel, kernel_size=1, stride=1, padding=0, bias=False)

#     def forward(self, x):
#         # [N, C, H , W]
#         b, c, w = x.size()
#         # [N, C/2, H * W]
# #         print(b, c, h, w)
# #         print(self.conv_phi(x).view(b, c, -1).shape)
#         x_phi = self.conv_phi(x).view(b,c, -1)
#         # [N, H * W, C/2]
#         x_theta = self.conv_theta(x).view(b, -1,c).contiguous()
#         x_g = self.conv_g(x).view(b, -1,c).contiguous()
# #         print(x_theta.shape)
# #         print(x_phi.shape)
# #         print(x_g.shape)
#         # [N, H * W, H * W]
#         mul_theta_phi = torch.matmul(x_theta, x_phi)
#         mul_theta_phi = self.softmax(mul_theta_phi)
#         # [N, H * W, C/2]
#         mul_theta_phi_g = torch.matmul(mul_theta_phi, x_g)
#         # [N, C/2, H, W]
#         mul_theta_phi_g = mul_theta_phi_g.permute(0,2,1).contiguous().view(b,self.inter_channel, w)
#         # [N, C, H , W]
#         mask = self.conv_mask(mul_theta_phi_g)
#         out = mask + x
#         return out
    
class Attention(nn.Module):
    def __init__(self):
        super(Attention, self).__init__()
        self.L = 128
        self.D = 256
        self.K = 1
        
        self.nl = NONLocalBlock1D(4)
        
#         self.nl=nn.Conv1d(in_channels=4, out_channels=4, kernel_size=1, stride=1,padding=0, bias=False)
        
        self.lossfn=nn.MSELoss()
        
        self.attention_transform_M = nn.Sequential(
            nn.Flatten(),
            nn.Linear(self.L*self.K*4, 512),
            nn.Tanh(),
            nn.Linear(512,self.K)
        )

        self.feature_extractor_HR = nn.Sequential(
#             nn.Linear(6,32),
#             nn.ReLU(),
            nn.Linear(6,self.D),
            nn.ReLU()
        )
        self.feature_extractor_EDA = nn.Sequential(
#             nn.Linear(6,32),
#             nn.ReLU(),
            nn.Linear(6,self.D),
            nn.ReLU()
        )
        self.feature_extractor_RSPR= nn.Sequential(
#             nn.Linear(6,32),
#             nn.ReLU(),
            nn.Linear(6,self.D),
            nn.ReLU()
        )
        self.feature_extractor_RSPA= nn.Sequential(
#             nn.Linear(6,16),
#             nn.ReLU(),
            nn.Linear(6,self.D),
            nn.ReLU()
        )
        
        self.feature_transform_HR = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(0.2),
            nn.BatchNorm1d(self.D, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True),
            nn.Linear(self.D, self.L)
        )
        
        self.attention_transform_HR = nn.Sequential(
            nn.Flatten(),
            nn.Linear(self.D, self.L),
            nn.Tanh(),
            nn.Linear(self.L,self.K)
        )
        
        self.feature_transform_EDA = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(0.1),
            nn.BatchNorm1d(self.D, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True),
            nn.Linear(self.D, self.L)
        )
        
        self.attention_transform_EDA = nn.Sequential(
            nn.Flatten(),
            nn.Linear(self.D, self.L),
            nn.Tanh(),
            nn.Linear(self.L,self.K)
        )
        
        self.feature_transform_RSPA = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(0.1),
            nn.BatchNorm1d(self.D, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True),
            nn.Linear(self.D, self.L)
        )
        
        self.attention_transform_RSPA = nn.Sequential(
            nn.Flatten(),
            nn.Linear(self.D, self.L),
            nn.Tanh(),
            nn.Linear(self.L,self.K)
        )
        
        self.feature_transform_RSPR = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(0.2),
            nn.BatchNorm1d(self.D, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True),
            nn.Linear(self.D, self.L)
        )
        
        self.attention_transform_RSPR = nn.Sequential(
            nn.Flatten(),
            nn.Linear(self.D, self.L),
            nn.Tanh(),
            nn.Linear(self.L,self.K)
        )

        self.classifier = nn.Sequential(
            nn.Linear(self.L*self.K*4, 256),
            nn.ReLU(),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )
        
    def attention_pooling(self,feature_embed_HR,feature_embed_EDA,feature_embed_RSPA,feature_embed_RSPR):
        
        #HR
        feature_transform_HR = self.feature_transform_HR(feature_embed_HR)
        
        A_t_hr = self.attention_transform_HR(feature_embed_HR)
        
        A_t_hr = torch.transpose(A_t_hr, 1, 0)
        
        A_t_hr = F.softmax(A_t_hr, dim=1)

        M_hr = torch.mm(A_t_hr, feature_transform_HR)
        
        #EDA
        
        feature_transform_EDA = self.feature_transform_HR(feature_embed_EDA)
        
        A_t_eda = self.attention_transform_EDA(feature_embed_EDA)
        
        A_t_eda = torch.transpose(A_t_eda, 1, 0)
        
        A_t_eda = F.softmax(A_t_eda, dim=1)

        M_eda = torch.mm(A_t_eda, feature_transform_EDA)
        
        #RSPA

        feature_transform_RSPA = self.feature_transform_RSPA(feature_embed_RSPA)
        
        A_t_rspa = self.attention_transform_RSPA(feature_embed_RSPA)
        
        A_t_rspa = torch.transpose(A_t_rspa, 1, 0)
        
        A_t_rspa = F.softmax(A_t_rspa, dim=1)

        M_rspa = torch.mm(A_t_rspa, feature_transform_RSPA)
        
        #RSPR

        feature_transform_RSPR = self.feature_transform_RSPR(feature_embed_RSPR)
        
        A_t_rspr = self.attention_transform_RSPR(feature_embed_RSPR)
        
        A_t_rspr = torch.transpose(A_t_rspr, 1, 0)
        
        A_t_rspr = F.softmax(A_t_rspr, dim=1)

        M_rspr = torch.mm(A_t_rspr, feature_transform_RSPR)
        
        M=torch.cat((M_hr,M_eda,M_rspa,M_rspr),dim=0)
        
        return M
        

    def forward(self, x):
        
        
        x=x.squeeze(0)
        
        #HR
        
        feature_embed_HR= self.feature_extractor_HR(x[:,:,:6])
        

        
        #EDA
        
        feature_embed_EDA= self.feature_extractor_EDA(x[:,:,6:12])
        
        
        #RSPA
        
        feature_embed_RSPA= self.feature_extractor_RSPA(x[:,:,12:18])
        
        
        #RSPR
        
        feature_embed_RSPR= self.feature_extractor_RSPR(x[:,:,18:24])
        
        
        
        M = self.attention_pooling(feature_embed_HR,feature_embed_EDA,feature_embed_RSPA,feature_embed_RSPR)
        
#         print(M.shape)
        
        M = M.reshape(1,4,self.L)
        
#         print(M.shape)
        
        M = self.nl(M) 
        
#         print(M.shape)
        
        M=M.reshape(M.size(0),-1)
        
#         print(M.shape)

        Y_prob = self.classifier(M)        
        
        Y_hat = torch.ge(Y_prob, 0.5).float()

        return Y_prob, Y_hat



    # AUXILIARY METHODS
    def calculate_classification_error(self, X, Y):
        Y = Y.float()
        _, Y_hat = self.forward(X)
        error = 1. - Y_hat.eq(Y).cpu().float().mean().item()

        return error, Y_hat

    def calculate_objective(self, X, Y):
        Y = Y.float()
        Y_prob, _ = self.forward(X)
        Y_prob = torch.clamp(Y_prob, min=1e-5, max=1. - 1e-5)
        loss=self.lossfn(Y_prob.squeeze(1),Y)
        
        return loss

In [6]:
model = Attention()
optimizer = optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.999))
# model.cuda()

/home/hsharm04/.conda/envs/physio/lib/python3.7/site-packages/ipykernel_launcher.py:40: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
/home/hsharm04/.conda/envs/physio/lib/python3.7/site-packages/ipykernel_launcher.py:41: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.


In [7]:
# from torchsummary import summary
# summary(model.cuda(),(1,24))

In [8]:
if torch.cuda.is_available():model.cuda()

In [9]:
best_model = "./saved_models/best_model_MIL_raw_fs_MH_fir"
train_acc = []
modelname=[]
val_acc = []
best_acc = 0
for epoch in range(0, 40):
    model.train()
    train_loss = 0.
    train_error = 0.
    y =[]
    ypred = []
    for batch_idx, (data, label) in enumerate(train_loader):
        bag_label = label[0]
        data, bag_label = data.cuda(), bag_label.cuda()
        data, bag_label = Variable(data), Variable(bag_label)
        optimizer.zero_grad()
        loss = model.calculate_objective(data.float(), bag_label.float())
        train_loss += loss.item()
        error,y_pred = model.calculate_classification_error(data.float(), bag_label.float())
        train_error += error
        y_pred=y_pred.squeeze(1)
        ypred.extend(y_pred.tolist())
        y.extend(bag_label.tolist())
        loss.backward()
        optimizer.step()
    
    trainacc = f1_score(y,ypred)
    train_loss /= len(train_loader)
    train_error /= len(train_loader)
    print("EPOCH ",epoch)
    print('Train : Loss: {:.4f}, Train error: {:.4f}, Train acc : {}'.format(train_loss, 
                                                                                train_error,trainacc))
    y =[]
    ypred = []
    model.eval()
    for batch_idx, (data, label) in enumerate(val_loader):
        bag_label = label[0]
        data, bag_label = data.cuda(), bag_label.cuda()
        data, bag_label = Variable(data), Variable(bag_label)
        error, y_pred = model.calculate_classification_error(data.float(), bag_label.float())
        y_pred=y_pred.squeeze(1)
        ypred.extend(y_pred.tolist())
        y.extend(bag_label.tolist())
    valacc = f1_score(y,ypred)
    print('Val acc :{}'.format(valacc))
    if (valacc>=best_acc and trainacc>0.65):
        print("---------State saved---------")
        best_acc = valacc
        best_state=model.state_dict()
        torch.save(best_state, best_model+'_epoch_'+str(epoch)+".pth")
        modelname.append(best_model+'_epoch_'+str(epoch)+".pth")
    print('Best validation accuracy ',best_acc)

    y =[]
    ypred = []
    for batch_idx, (data, label) in enumerate(test_loader):
        bag_label = label[0]
        data, bag_label = data.cuda(), bag_label.cuda()
        data, bag_label = Variable(data), Variable(bag_label)
        error, y_pred = model.calculate_classification_error(data.float(), bag_label.float())
        y_pred=y_pred.squeeze(1)
        ypred.extend(y_pred.tolist())
        y.extend(bag_label.tolist())
    valacc = f1_score(y,ypred)
    print('test acc :{}'.format(valacc))
        

EPOCH  0
Train : Loss: 0.2441, Train error: 0.4228, Train acc : 0.6353846153846153
Val acc :0.5531914893617021
Best validation accuracy  0
test acc :0.4588235294117647
EPOCH  1
Train : Loss: 0.2379, Train error: 0.3880, Train acc : 0.679439941046426
Val acc :0.5541125541125541
---------State saved---------
Best validation accuracy  0.5541125541125541
test acc :0.5502183406113537
EPOCH  2
Train : Loss: 0.2307, Train error: 0.3684, Train acc : 0.697878566203365
Val acc :0.5421686746987953
Best validation accuracy  0.5541125541125541
test acc :0.40506329113924056
EPOCH  3
Train : Loss: 0.2273, Train error: 0.3747, Train acc : 0.6645367412140575
Val acc :0.5625
---------State saved---------
Best validation accuracy  0.5625
test acc :0.34210526315789475
EPOCH  4
Train : Loss: 0.2198, Train error: 0.3613, Train acc : 0.6848249027237354
Val acc :0.5026737967914437
Best validation accuracy  0.5625
test acc :0.4971751412429378
EPOCH  5
Train : Loss: 0.2087, Train error: 0.3327, Train acc : 0.70

In [11]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
device = torch.device("cuda")
model=Attention()
# best_state=torch.load("../best-models/Attention_MIL_best(stress)_F1_91.pth")
best_state=torch.load(modelname[-1])
model.load_state_dict(best_state)
model.to(device)
model.eval()
test_loss = 0.
correct = 0
total = 0
y =[]
ypred = []
test_error = 0.
for batch_idx, (data, label) in enumerate(test_loader):
    bag_label = label[0]
    instance_labels = label[1]
    data, bag_label = data.cuda(), bag_label.cuda()
    data, bag_label = Variable(data), Variable(bag_label)
    loss = model.calculate_objective(data.float(), bag_label.float())
    test_loss += loss.item()
    error, predicted_label = model.calculate_classification_error(data.float(), bag_label.float())
    test_error += error
    predicted_label=predicted_label.squeeze(1)
    ypred.extend(predicted_label.tolist())
    y.extend(bag_label.tolist())
    
acc=accuracy_score(y,ypred)
# print(y,ypred)
tn, fp, fn, tp = confusion_matrix(y,ypred).ravel()
f1score=f1_score(y,ypred)
precision=precision_score(y,ypred)
recall=recall_score(y,ypred)
roc=roc_auc_score(y,ypred)
specificity=tn/(tn+fp)
print(acc,f1score,precision,recall,roc,specificity)

for i in modelname:

    model=Attention()
    # best_state=torch.load("../best-models/Attention_MIL_best(stress)_F1_91.pth")
    best_state=torch.load(i)
    model.load_state_dict(best_state)
    model.to(device)
    model.eval()
    test_loss = 0.
    correct = 0
    total = 0
    y =[]
    ypred = []
    test_error = 0.
    for batch_idx, (data, label) in enumerate(test_loader):
        bag_label = label[0]
        instance_labels = label[1]
        data, bag_label = data.cuda(), bag_label.cuda()
        data, bag_label = Variable(data), Variable(bag_label)
        loss = model.calculate_objective(data.float(), bag_label.float())
        test_loss += loss.item()
        error, predicted_label = model.calculate_classification_error(data.float(), bag_label.float())
        test_error += error
        predicted_label=predicted_label.squeeze(1)
        ypred.extend(predicted_label.tolist())
        y.extend(bag_label.tolist())

    acc=accuracy_score(y,ypred)
    # print(y,ypred)
    tn, fp, fn, tp = confusion_matrix(y,ypred).ravel()
    f1score=f1_score(y,ypred)
    precision=precision_score(y,ypred)
    recall=recall_score(y,ypred)
    roc=roc_auc_score(y,ypred)
    specificity=tn/(tn+fp)
    print(acc,f1score,precision,recall,roc,specificity)


/home/hsharm04/.conda/envs/physio/lib/python3.7/site-packages/ipykernel_launcher.py:40: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
/home/hsharm04/.conda/envs/physio/lib/python3.7/site-packages/ipykernel_launcher.py:41: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.


0.8442211055276382 0.822857142857143 0.8470588235294118 0.8 0.8403669724770643 0.8807339449541285


/home/hsharm04/.conda/envs/physio/lib/python3.7/site-packages/ipykernel_launcher.py:40: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
/home/hsharm04/.conda/envs/physio/lib/python3.7/site-packages/ipykernel_launcher.py:41: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.


0.4824120603015075 0.5502183406113537 0.45323741007194246 0.7 0.5013761467889908 0.30275229357798167


/home/hsharm04/.conda/envs/physio/lib/python3.7/site-packages/ipykernel_launcher.py:40: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
/home/hsharm04/.conda/envs/physio/lib/python3.7/site-packages/ipykernel_launcher.py:41: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.


...

0.9296482412060302 0.9222222222222223 0.9222222222222223 0.9222222222222223 0.9290010193679917 0.9357798165137615


/home/hsharm04/.conda/envs/physio/lib/python3.7/site-packages/ipykernel_launcher.py:40: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
/home/hsharm04/.conda/envs/physio/lib/python3.7/site-packages/ipykernel_launcher.py:41: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.


0.864321608040201 0.8457142857142856 0.8705882352941177 0.8222222222222222 0.8606523955147808 0.8990825688073395


/home/hsharm04/.conda/envs/physio/lib/python3.7/site-packages/ipykernel_launcher.py:40: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
/home/hsharm04/.conda/envs/physio/lib/python3.7/site-packages/ipykernel_launcher.py:41: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.


0.8442211055276382 0.822857142857143 0.8470588235294118 0.8 0.8403669724770643 0.8807339449541285


In [12]:
modelname[-1]

'./saved_models/best_model_MIL_raw_fs_MH_fir_epoch_39.pth'